In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt 
import pymssql
import seaborn as sns
import sklearn
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.model_selection import train_test_split 
from xgboost import XGBRegressor
from sklearn import metrics

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def get_data():
    df = pd.DataFrame()
    try:
        con = pymssql.connect(user='username',password = 'password'
                 ,host='server',database='db',autocommit = True)
    except Exception as e:
        print('unable to make connection',e)
    cur=con.cursor()

    query = """
    -------------------------------------PAYMENTS AND CHARGES DATA----------------------------------
    ----------------------------charges data--------------------------------------
    IF OBJECT_ID('TEMPDB..#RCM_CHA') IS NOT NULL
    DROP TABLE #RCM_CHA
    Select pr.practice_code as Practice_Code,  MONTH(cc.DOE) as [Month], year(cc.DOE) as [Year],
    SUM(cc.Amount) as Charges_Sum
    into #RCM_CHA from Claim_Charges cc 
    join Claims c on
    cc.Claim_No = c.Claim_No and DATEDIFF(day, cc.DOE, GETDATE())<800
    and isnull(c.Deleted,0)= 0 and isnull(cc.Deleted,0)= 0
    join Patient p on
    p.Patient_Account = c.Patient_Account
    and isnull(p.Deleted,0)= 0
    join PRACTICES pr on
    pr.Practice_Code = p.Practice_Code 
    AND ISNULL(P.DELETED,0)=0
    AND PR.IS_ACTIVE=1 AND ISNULL(PR.IS_TEST_PRACTICE,0)=0
    AND ISNULL(PR.EXCLUDE_FROM_BILLING_REPORTS,0) = 0
    AND PR.EMR_NAME NOT IN ('CHARTS PRO','PM STANDALONE','TESTINGPRACTICE')
    AND PR.PRACTICE_CODE NOT LIKE '9090%' AND PR.PRAC_NAME NOT LIKE '%TEST%'
    AND ISNULL(Pr.Deleted,0)=0
    group by pr.practice_code , MONTH(cc.DOE), YEAR(cc.DOE)
    -----------------------------------------------------------------------  
    ---------------------------------Payments data--------------------------------------
    IF OBJECT_ID('TEMPDB..#RCM_PAY') IS NOT NULL
    DROP TABLE #RCM_PAY
    Select pr.practice_code as Practice_Code,  MONTH(CP.Date_Entry) as [Month], year(CP.Date_Entry) as [Year],
    SUM(cp.Amount_Paid) as Paymnet_Sum
    into #RCM_PAY from Claim_Payments cp
    join Claims c on
    cP.Claim_No  = c.Claim_No and DATEDIFF(day, CP.Date_Entry, GETDATE())<800
    and isnull(c.Deleted,0)= 0 and isnull(cp.Deleted,0)= 0
    join Patient p on
    p.Patient_Account = c.Patient_Account
    and isnull(p.Deleted,0)= 0
    join PRACTICES pr on
    pr.Practice_Code = p.Practice_Code 
    AND ISNULL(P.DELETED,0)=0
    AND PR.IS_ACTIVE=1 AND ISNULL(PR.IS_TEST_PRACTICE,0)=0
    AND ISNULL(PR.EXCLUDE_FROM_BILLING_REPORTS,0) = 0
    AND PR.EMR_NAME NOT IN ('CHARTS PRO','PM STANDALONE','TESTINGPRACTICE')
    AND PR.PRACTICE_CODE NOT LIKE '9090%' AND PR.PRAC_NAME NOT LIKE '%TEST%'
    AND ISNULL(Pr.Deleted,0)=0
    group by pr.practice_code , MONTH(cp.Date_Entry), YEAR(CP.Date_Entry)
    -----------------------------------------------------------------------
    IF OBJECT_ID('TEMPDB..#RCM_PCD') IS NOT NULL
    DROP TABLE #RCM_PCD
    Select C.practice_code, C.[Month],C.[Year],C.Charges_Sum,p.Paymnet_Sum
    into #RCM_PCD
    from #RCM_CHA C
    join #RCM_PAY P on
    c.Practice_Code = p.Practice_Code and c.[Month] = p.[Month] and c.[Year] = p.[Year]  
    -----------------------------------------------------------------------------------------------
    IF OBJECT_ID('TEMPDB..#RCM_DATA') IS NOT NULL
    DROP TABLE #RCM_DATA
    SELECT r.practice_code, r.[Month],r.[Year],r.Charges_Sum,r.Paymnet_Sum,
    a.AGING_0_30, a.AGING_31_60, a.AGING_61_90, a.AGING_91_120,a.AGING_120_PLUS  into #RCM_DATA
    from #RCM_PCD r
    join MONTH_END_AGING a on
    r.practice_code = a.PRACTICE_CODE and r.[Year] = a.CLOSE_YEAR and  r.[Month] = a.CLOSE_MONTH 
    order by practice_code, YEAR, MONTH 
	------------------------------------------------------------------------------------------------------------
   	IF OBJECT_ID('TEMPDB..#RCM_PREV') IS NOT NULL
    DROP TABLE #RCM_PREV
    SELECT practice_code, Month,Year,Charges_Sum,
    COALESCE(lag(Charges_Sum) over (partition by PRACTICE_CODE order by [YEAR],[MONTH]), 0) as Prev_Month_Charges,
    Paymnet_Sum, 
    COALESCE(lag(Paymnet_Sum) over (partition by PRACTICE_CODE order by [YEAR],[MONTH]), 0) as Prev_Month_Payments,
    AGING_0_30, AGING_31_60, AGING_61_90, AGING_91_120,AGING_120_PLUS  into #RCM_PREV
    from #RCM_DATA
    -----------------------------------------------------------------------------------------------------------------
   	IF OBJECT_ID('TEMPDB..#RCM_FIN') IS NOT NULL
    DROP TABLE #RCM_FIN
    SELECT practice_code, Month,Year,Charges_Sum,Prev_Month_Charges,
    COALESCE(lag(Prev_Month_Charges) over (partition by PRACTICE_CODE order by [YEAR],[MONTH]), 0) as Prev_2nd_Month_Charges,
    Paymnet_Sum, Prev_Month_Payments,
    COALESCE(lag(Prev_Month_Payments) over (partition by PRACTICE_CODE order by [YEAR],[MONTH]), 0) as Prev_2nd_Month_Payments,
    AGING_0_30, AGING_31_60, AGING_61_90, AGING_91_120,AGING_120_PLUS  into #RCM_FIN
    from #RCM_PREV
    
    Select * from #RCM_FIN order by practice_code, YEAR, MONTH 
    ---------------------------------------------------------------------------------------------------------------------------

    
     
    """ 
    
    try:
        cur.execute(query)
        df=pd.DataFrame(cur.fetchall(),columns=[x[0] for x in cur.description])
        df.columns=df.columns.str.upper()
        con.close()
    except:
        print('Error in reading data.....')
        con.close()
    return df

In [28]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [29]:
# df = get_data()

In [30]:
# df.to_csv("YEARLY_PAY_CHAR.CSV", index = False)

In [61]:
data = pd.read_csv("YEARLY_PAY_CHAR.CSV")

In [62]:
data_copy = data.copy()

In [63]:
data.head(50)

,PRACTICE_CODE,MONTH,YEAR,CHARGES_SUM,PREV_MONTH_CHARGES,PREV_2ND_MONTH_CHARGES,PAYMNET_SUM,PREV_MONTH_PAYMENTS,PREV_2ND_MONTH_PAYMENTS,AGING_0_30,AGING_31_60,AGING_61_90,AGING_91_120,AGING_120_PLUS
0,100,11,2020,9297.07,0.00,0.00,5108.11,0.00,0.00,38244.22,9652.18,3305.20,2550.99,4592.8302
1,100,12,2020,61761.44,9297.07,0.00,32226.04,5108.11,0.00,21375.85,10878.66,7961.48,2281.65,3254.9102
2,100,1,2021,79764.30,61761.44,9297.07,26264.63,32226.04,5108.11,42863.04,3050.95,7847.44,3207.22,3997.2602
3,100,2,2021,76536.02,79764.30,61761.44,31416.99,26264.63,32226.04,39047.64,7894.41,2460.04,3188.86,4775.6502
4,100,3,2021,47048.07,76536.02,79764.30,25210.00,31416.99,26264.63,17126.02,8333.23,5692.38,941.79,5408.3902
5,100,4,2021,100579.09,47048.07,76536.02,28978.45,25210.00,31416.99,44560.57,5197.07,5167.59,3507.86,4092.2202
6,100,5,2021,92388.12,100579.09,47048.07,36637.56,28978.45,25210.00,40315.12,8663.09,2110.69,2627.38,3788.6202
7,100,6,2021,113708.04,92388.12,100579.09,37826.63,36637.56,28978.45,51354.02,7460.24,5643.48,751.78,3009.9302
8,100,7,2021,81952.53,113708.04,92388.12,41238.33,37826.63,36637.56,27547.96,6815.34,4060.63,3455.76,3604.7202
9,100,8,2021,108765.04,81952.53,113708.04,34942.13,41238.33,37826.63,49712.61,3617.32,4913.80,2418.82,4628.4902


In [64]:
data['PC_RATIO'] = (data['PAYMNET_SUM']/data['CHARGES_SUM'])*100

In [65]:
data.fillna(0, inplace=True)
data.replace([np.inf, -np.inf], 0, inplace=True)

In [66]:
data = data.rename(columns = {'PRACTICE_CODE':'PRAC_CODE', 'CHARGES_SUM':'CHARG_SUM','PREV_MONTH_CHARGES':'PREV_M_CHARG','PREV_2ND_MONTH_CHARGES':'PREV_2ND_M_CHARG',
                      'PAYMNET_SUM':'PAY_SUM','PREV_MONTH_PAYMENTS':'PREV_M_PAY','PREV_2ND_MONTH_PAYMENTS':'PREV_2ND_M_PAY'})

In [67]:
# data.columns

In [68]:
data['PREV_3RD_M_CHARG'] = data['PREV_2ND_M_CHARG'].shift(1)
data['PREV_4TH_M_CHARG'] = data['PREV_3RD_M_CHARG'].shift(1)
data['PREV_5TH_M_CHARG'] = data['PREV_4TH_M_CHARG'].shift(1)
data['PREV_6TH_M_CHARG'] = data['PREV_5TH_M_CHARG'].shift(1)
data['PREV_7TH_M_CHARG'] = data['PREV_6TH_M_CHARG'].shift(1)
data['PREV_8TH_M_CHARG'] = data['PREV_7TH_M_CHARG'].shift(1)
data['PREV_9TH_M_CHARG'] = data['PREV_8TH_M_CHARG'].shift(1)
data['PREV_10TH_M_CHARG'] = data['PREV_9TH_M_CHARG'].shift(1)
data['PREV_11TH_M_CHARG'] = data['PREV_10TH_M_CHARG'].shift(1)
data['PREV_12TH_M_CHARG'] = data['PREV_11TH_M_CHARG'].shift(1)

In [69]:
data['PREV_3RD_M_PAY'] = data['PREV_2ND_M_PAY'].shift(1)
data['PREV_4TH_M_PAY'] = data['PREV_3RD_M_PAY'].shift(1)
data['PREV_5TH_M_PAY'] = data['PREV_4TH_M_PAY'].shift(1)
data['PREV_6TH_M_PAY'] = data['PREV_5TH_M_PAY'].shift(1)
data['PREV_7TH_M_PAY'] = data['PREV_6TH_M_PAY'].shift(1)
data['PREV_8TH_M_PAY'] = data['PREV_7TH_M_PAY'].shift(1)
data['PREV_9TH_M_PAY'] = data['PREV_8TH_M_PAY'].shift(1)
data['PREV_10TH_M_PAY'] = data['PREV_9TH_M_PAY'].shift(1)
data['PREV_11TH_M_PAY'] = data['PREV_10TH_M_PAY'].shift(1)
data['PREV_12TH_M_PAY'] = data['PREV_11TH_M_PAY'].shift(1)

In [70]:
data['PREV_M_AGING_0_30'] = data['AGING_0_30'].shift(1)
data['PREV_2M_AGING_0_30'] = data['PREV_M_AGING_0_30'].shift(1)
data['PREV_3M_AGING_0_30'] = data['PREV_2M_AGING_0_30'].shift(1)
data['PREV_4M_AGING_0_30'] = data['PREV_3M_AGING_0_30'].shift(1)
data['PREV_5M_AGING_0_30'] = data['PREV_4M_AGING_0_30'].shift(1)
data['PREV_6M_AGING_0_30'] = data['PREV_5M_AGING_0_30'].shift(1)
data['PREV_7M_AGING_0_30'] = data['PREV_6M_AGING_0_30'].shift(1)
data['PREV_8M_AGING_0_30'] = data['PREV_7M_AGING_0_30'].shift(1)
data['PREV_9M_AGING_0_30'] = data['PREV_8M_AGING_0_30'].shift(1)
data['PREV_10M_AGING_0_30'] = data['PREV_9M_AGING_0_30'].shift(1)
data['PREV_11M_AGING_0_30'] = data['PREV_10M_AGING_0_30'].shift(1)
data['PREV_12M_AGING_0_30'] = data['PREV_11M_AGING_0_30'].shift(1)

In [71]:
data['PREV_M_AGING_31_60'] = data['AGING_31_60'].shift(1)
data['PREV_2M_AGING_31_60'] = data['PREV_M_AGING_31_60'].shift(1)
data['PREV_3M_AGING_31_60'] = data['PREV_2M_AGING_31_60'].shift(1)
data['PREV_4M_AGING_31_60'] = data['PREV_3M_AGING_31_60'].shift(1)
data['PREV_5M_AGING_31_60'] = data['PREV_4M_AGING_31_60'].shift(1)
data['PREV_6M_AGING_31_60'] = data['PREV_5M_AGING_31_60'].shift(1)
data['PREV_7M_AGING_31_60'] = data['PREV_6M_AGING_31_60'].shift(1)
data['PREV_8M_AGING_31_60'] = data['PREV_7M_AGING_31_60'].shift(1)
data['PREV_9M_AGING_31_60'] = data['PREV_8M_AGING_31_60'].shift(1)
data['PREV_10M_AGING_31_60'] = data['PREV_9M_AGING_31_60'].shift(1)
data['PREV_11M_AGING_31_60'] = data['PREV_10M_AGING_31_60'].shift(1)
data['PREV_12M_AGING_31_60'] = data['PREV_11M_AGING_31_60'].shift(1)

In [72]:
data['PREV_M_AGING_61_90'] = data.groupby('PRAC_CODE')['AGING_61_90'].shift(1)
data['PREV_2M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_M_AGING_61_90'].shift(1)
data['PREV_3M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_2M_AGING_61_90'].shift(1)
data['PREV_4M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_3M_AGING_61_90'].shift(1)
data['PREV_5M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_4M_AGING_61_90'].shift(1)
data['PREV_6M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_5M_AGING_61_90'].shift(1)
data['PREV_7M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_6M_AGING_61_90'].shift(1)
data['PREV_8M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_7M_AGING_61_90'].shift(1)
data['PREV_9M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_8M_AGING_61_90'].shift(1)
data['PREV_10M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_9M_AGING_61_90'].shift(1)
data['PREV_11M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_10M_AGING_61_90'].shift(1)
data['PREV_12M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_11M_AGING_61_90'].shift(1)

In [73]:
data['PREV_M_AGING_91_120'] = data['AGING_91_120'].shift(1)
data['PREV_2M_AGING_91_120'] = data['PREV_M_AGING_91_120'].shift(1)
data['PREV_3M_AGING_91_120'] = data['PREV_2M_AGING_91_120'].shift(1)
data['PREV_4M_AGING_91_120'] = data['PREV_3M_AGING_91_120'].shift(1)
data['PREV_5M_AGING_91_120'] = data['PREV_4M_AGING_91_120'].shift(1)
data['PREV_6M_AGING_91_120'] = data['PREV_5M_AGING_91_120'].shift(1)
data['PREV_7M_AGING_91_120'] = data['PREV_6M_AGING_91_120'].shift(1)
data['PREV_8M_AGING_91_120'] = data['PREV_7M_AGING_91_120'].shift(1)
data['PREV_9M_AGING_91_120'] = data['PREV_8M_AGING_91_120'].shift(1)
data['PREV_10M_AGING_91_120'] = data['PREV_9M_AGING_91_120'].shift(1)
data['PREV_11M_AGING_91_120'] = data['PREV_10M_AGING_91_120'].shift(1)
data['PREV_12M_AGING_91_120'] = data['PREV_11M_AGING_91_120'].shift(1)

In [74]:
data['PREV_M_AGING_120_PLUS'] = data['AGING_120_PLUS'].shift(1)
data['PREV_2M_AGING_120_PLUS'] = data['PREV_M_AGING_120_PLUS'].shift(1)
data['PREV_3M_AGING_120_PLUS'] = data['PREV_2M_AGING_120_PLUS'].shift(1)
data['PREV_4M_AGING_120_PLUS'] = data['PREV_3M_AGING_120_PLUS'].shift(1)
data['PREV_5M_AGING_120_PLUS'] = data['PREV_4M_AGING_120_PLUS'].shift(1)
data['PREV_6M_AGING_120_PLUS'] = data['PREV_5M_AGING_120_PLUS'].shift(1)
data['PREV_7M_AGING_120_PLUS'] = data['PREV_6M_AGING_120_PLUS'].shift(1)
data['PREV_8M_AGING_120_PLUS'] = data['PREV_7M_AGING_120_PLUS'].shift(1)
data['PREV_9M_AGING_120_PLUS'] = data['PREV_8M_AGING_120_PLUS'].shift(1)
data['PREV_10M_AGING_120_PLUS'] = data['PREV_9M_AGING_120_PLUS'].shift(1)
data['PREV_11M_AGING_120_PLUS'] = data['PREV_10M_AGING_120_PLUS'].shift(1)
data['PREV_12M_AGING_120_PLUS'] = data['PREV_11M_AGING_120_PLUS'].shift(1)

In [75]:
data['PREV_M_PC_RATIO'] = data['PC_RATIO'].shift(1)
data['PREV_2M_PC_RATIO'] = data['PREV_M_PC_RATIO'].shift(1)
data['PREV_3M_PC_RATIO'] = data['PREV_2M_PC_RATIO'].shift(1)
data['PREV_4M_PC_RATIO'] = data['PREV_3M_PC_RATIO'].shift(1)
data['PREV_5M_PC_RATIO'] = data['PREV_4M_PC_RATIO'].shift(1)
data['PREV_6M_PC_RATIO'] = data['PREV_5M_PC_RATIO'].shift(1)
data['PREV_7M_PC_RATIO'] = data['PREV_6M_PC_RATIO'].shift(1)
data['PREV_8M_PC_RATIO'] = data['PREV_7M_PC_RATIO'].shift(1)
data['PREV_9M_PC_RATIO'] = data['PREV_8M_PC_RATIO'].shift(1)
data['PREV_10M_PC_RATIO'] = data['PREV_9M_PC_RATIO'].shift(1)
data['PREV_11M_PC_RATIO'] = data['PREV_10M_PC_RATIO'].shift(1)
data['PREV_12M_PC_RATIO'] = data['PREV_11M_PC_RATIO'].shift(1)

In [185]:
data.head(1)

,PRAC_CODE,MONTH,YEAR,CHARG_SUM,PREV_M_CHARG,PREV_2ND_M_CHARG,PREV_3RD_M_CHARG,PREV_4TH_M_CHARG,PREV_5TH_M_CHARG,PREV_6TH_M_CHARG,PREV_7TH_M_CHARG,PREV_8TH_M_CHARG,PREV_9TH_M_CHARG,PREV_10TH_M_CHARG,PREV_11TH_M_CHARG,PREV_12TH_M_CHARG,PAY_SUM,PREV_M_PAY,PREV_2ND_M_PAY,PREV_3RD_M_PAY,PREV_4TH_M_PAY,PREV_5TH_M_PAY,PREV_6TH_M_PAY,PREV_7TH_M_PAY,PREV_8TH_M_PAY,PREV_9TH_M_PAY,PREV_10TH_M_PAY,PREV_11TH_M_PAY,PREV_12TH_M_PAY,AGING_0_30,PREV_M_AGING_0_30,PREV_2M_AGING_0_30,PREV_3M_AGING_0_30,PREV_4M_AGING_0_30,PREV_5M_AGING_0_30,PREV_6M_AGING_0_30,PREV_7M_AGING_0_30,PREV_8M_AGING_0_30,PREV_9M_AGING_0_30,PREV_10M_AGING_0_30,PREV_11M_AGING_0_30,PREV_12M_AGING_0_30,AGING_31_60,PREV_M_AGING_31_60,PREV_2M_AGING_31_60,PREV_3M_AGING_31_60,PREV_4M_AGING_31_60,PREV_5M_AGING_31_60,PREV_6M_AGING_31_60,PREV_7M_AGING_31_60,PREV_8M_AGING_31_60,PREV_9M_AGING_31_60,PREV_10M_AGING_31_60,PREV_11M_AGING_31_60,PREV_12M_AGING_31_60,AGING_61_90,PREV_M_AGING_61_90,PREV_2M_AGING_61_90,PREV_3M_AGING_61_90,PREV_4M_AGING_61_90,PREV_5M_AGING_61_90,PREV_6M_AGING_61_90,PREV_7M_AGING_61_90,PREV_8M_AGING_61_90,PREV_9M_AGING_61_90,PREV_10M_AGING_61_90,PREV_11M_AGING_61_90,PREV_12M_AGING_61_90,AGING_91_120,PREV_M_AGING_91_120,PREV_2M_AGING_91_120,PREV_3M_AGING_91_120,PREV_4M_AGING_91_120,PREV_5M_AGING_91_120,PREV_6M_AGING_91_120,PREV_7M_AGING_91_120,PREV_8M_AGING_91_120,PREV_9M_AGING_91_120,PREV_10M_AGING_91_120,PREV_11M_AGING_91_120,PREV_12M_AGING_91_120,AGING_120_PLUS,PREV_M_AGING_120_PLUS,PREV_2M_AGING_120_PLUS,PREV_3M_AGING_120_PLUS,PREV_4M_AGING_120_PLUS,PREV_5M_AGING_120_PLUS,PREV_6M_AGING_120_PLUS,PREV_7M_AGING_120_PLUS,PREV_8M_AGING_120_PLUS,PREV_9M_AGING_120_PLUS,PREV_10M_AGING_120_PLUS,PREV_11M_AGING_120_PLUS,PREV_12M_AGING_120_PLUS,PREV_M_PC_RATIO,PREV_2M_PC_RATIO,PREV_3M_PC_RATIO,PREV_4M_PC_RATIO,PREV_5M_PC_RATIO,PREV_6M_PC_RATIO,PREV_7M_PC_RATIO,PREV_8M_PC_RATIO,PREV_9M_PC_RATIO,PREV_10M_PC_RATIO,PREV_11M_PC_RATIO,PREV_12M_PC_RATIO
0,100,11,2020,9297.07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5108.11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38244.22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9652.18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3305.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2550.99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4592.8302,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [190]:
data = data.loc[data['YEAR']>2021]

In [191]:
# data11.head()

In [192]:
data = data[['PRAC_CODE','MONTH','YEAR','CHARG_SUM','PREV_M_CHARG','PREV_2ND_M_CHARG','PREV_3RD_M_CHARG','PREV_4TH_M_CHARG',
'PREV_5TH_M_CHARG','PREV_6TH_M_CHARG', 'PREV_7TH_M_CHARG','PREV_8TH_M_CHARG', 'PREV_9TH_M_CHARG', 'PREV_10TH_M_CHARG',
'PREV_11TH_M_CHARG', 'PREV_12TH_M_CHARG','PAY_SUM', 'PREV_M_PAY', 'PREV_2ND_M_PAY','PREV_3RD_M_PAY',
'PREV_4TH_M_PAY', 'PREV_5TH_M_PAY', 'PREV_6TH_M_PAY', 'PREV_7TH_M_PAY',
'PREV_8TH_M_PAY', 'PREV_9TH_M_PAY', 'PREV_10TH_M_PAY','PREV_11TH_M_PAY', 'PREV_12TH_M_PAY', 'AGING_0_30','PREV_M_AGING_0_30',
'PREV_2M_AGING_0_30', 'PREV_3M_AGING_0_30', 'PREV_4M_AGING_0_30','PREV_5M_AGING_0_30', 'PREV_6M_AGING_0_30',
'PREV_7M_AGING_0_30','PREV_8M_AGING_0_30', 'PREV_9M_AGING_0_30', 'PREV_10M_AGING_0_30','PREV_11M_AGING_0_30',
'PREV_12M_AGING_0_30','AGING_31_60','PREV_M_AGING_31_60','PREV_2M_AGING_31_60', 'PREV_3M_AGING_31_60', 'PREV_4M_AGING_31_60',
'PREV_5M_AGING_31_60', 'PREV_6M_AGING_31_60', 'PREV_7M_AGING_31_60','PREV_8M_AGING_31_60', 'PREV_9M_AGING_31_60',
'PREV_10M_AGING_31_60','PREV_11M_AGING_31_60', 'PREV_12M_AGING_31_60','AGING_61_90',
'PREV_M_AGING_61_90', 'PREV_2M_AGING_61_90', 'PREV_3M_AGING_61_90', 'PREV_4M_AGING_61_90', 'PREV_5M_AGING_61_90',
'PREV_6M_AGING_61_90', 'PREV_7M_AGING_61_90', 'PREV_8M_AGING_61_90', 'PREV_9M_AGING_61_90', 'PREV_10M_AGING_61_90',
'PREV_11M_AGING_61_90', 'PREV_12M_AGING_61_90', 'AGING_91_120', 'PREV_M_AGING_91_120',
'PREV_2M_AGING_91_120', 'PREV_3M_AGING_91_120', 'PREV_4M_AGING_91_120','PREV_5M_AGING_91_120',
'PREV_6M_AGING_91_120','PREV_7M_AGING_91_120','PREV_8M_AGING_91_120', 'PREV_9M_AGING_91_120', 
'PREV_10M_AGING_91_120','PREV_11M_AGING_91_120','PREV_12M_AGING_91_120','AGING_120_PLUS',
'PREV_M_AGING_120_PLUS', 'PREV_2M_AGING_120_PLUS','PREV_3M_AGING_120_PLUS', 'PREV_4M_AGING_120_PLUS',
'PREV_5M_AGING_120_PLUS', 'PREV_6M_AGING_120_PLUS','PREV_7M_AGING_120_PLUS', 'PREV_8M_AGING_120_PLUS',
'PREV_9M_AGING_120_PLUS', 'PREV_10M_AGING_120_PLUS','PREV_11M_AGING_120_PLUS', 'PREV_12M_AGING_120_PLUS',
'PREV_M_PC_RATIO','PREV_2M_PC_RATIO', 'PREV_3M_PC_RATIO', 'PREV_4M_PC_RATIO','PREV_5M_PC_RATIO', 
'PREV_6M_PC_RATIO', 'PREV_7M_PC_RATIO','PREV_8M_PC_RATIO', 'PREV_9M_PC_RATIO', 'PREV_10M_PC_RATIO',
'PREV_11M_PC_RATIO', 'PREV_12M_PC_RATIO']]

In [193]:
data.fillna(0, inplace=True)
data.replace([np.inf, -np.inf], 0, inplace=True)

In [194]:
data.head()

,PRAC_CODE,MONTH,YEAR,CHARG_SUM,PREV_M_CHARG,PREV_2ND_M_CHARG,PREV_3RD_M_CHARG,PREV_4TH_M_CHARG,PREV_5TH_M_CHARG,PREV_6TH_M_CHARG,PREV_7TH_M_CHARG,PREV_8TH_M_CHARG,PREV_9TH_M_CHARG,PREV_10TH_M_CHARG,PREV_11TH_M_CHARG,PREV_12TH_M_CHARG,PAY_SUM,PREV_M_PAY,PREV_2ND_M_PAY,PREV_3RD_M_PAY,PREV_4TH_M_PAY,PREV_5TH_M_PAY,PREV_6TH_M_PAY,PREV_7TH_M_PAY,PREV_8TH_M_PAY,PREV_9TH_M_PAY,PREV_10TH_M_PAY,PREV_11TH_M_PAY,PREV_12TH_M_PAY,AGING_0_30,PREV_M_AGING_0_30,PREV_2M_AGING_0_30,PREV_3M_AGING_0_30,PREV_4M_AGING_0_30,PREV_5M_AGING_0_30,PREV_6M_AGING_0_30,PREV_7M_AGING_0_30,PREV_8M_AGING_0_30,PREV_9M_AGING_0_30,PREV_10M_AGING_0_30,PREV_11M_AGING_0_30,PREV_12M_AGING_0_30,AGING_31_60,PREV_M_AGING_31_60,PREV_2M_AGING_31_60,PREV_3M_AGING_31_60,PREV_4M_AGING_31_60,PREV_5M_AGING_31_60,PREV_6M_AGING_31_60,PREV_7M_AGING_31_60,PREV_8M_AGING_31_60,PREV_9M_AGING_31_60,PREV_10M_AGING_31_60,PREV_11M_AGING_31_60,PREV_12M_AGING_31_60,AGING_61_90,PREV_M_AGING_61_90,PREV_2M_AGING_61_90,PREV_3M_AGING_61_90,PREV_4M_AGING_61_90,PREV_5M_AGING_61_90,PREV_6M_AGING_61_90,PREV_7M_AGING_61_90,PREV_8M_AGING_61_90,PREV_9M_AGING_61_90,PREV_10M_AGING_61_90,PREV_11M_AGING_61_90,PREV_12M_AGING_61_90,AGING_91_120,PREV_M_AGING_91_120,PREV_2M_AGING_91_120,PREV_3M_AGING_91_120,PREV_4M_AGING_91_120,PREV_5M_AGING_91_120,PREV_6M_AGING_91_120,PREV_7M_AGING_91_120,PREV_8M_AGING_91_120,PREV_9M_AGING_91_120,PREV_10M_AGING_91_120,PREV_11M_AGING_91_120,PREV_12M_AGING_91_120,AGING_120_PLUS,PREV_M_AGING_120_PLUS,PREV_2M_AGING_120_PLUS,PREV_3M_AGING_120_PLUS,PREV_4M_AGING_120_PLUS,PREV_5M_AGING_120_PLUS,PREV_6M_AGING_120_PLUS,PREV_7M_AGING_120_PLUS,PREV_8M_AGING_120_PLUS,PREV_9M_AGING_120_PLUS,PREV_10M_AGING_120_PLUS,PREV_11M_AGING_120_PLUS,PREV_12M_AGING_120_PLUS,PREV_M_PC_RATIO,PREV_2M_PC_RATIO,PREV_3M_PC_RATIO,PREV_4M_PC_RATIO,PREV_5M_PC_RATIO,PREV_6M_PC_RATIO,PREV_7M_PC_RATIO,PREV_8M_PC_RATIO,PREV_9M_PC_RATIO,PREV_10M_PC_RATIO,PREV_11M_PC_RATIO,PREV_12M_PC_RATIO
11,100,1,2022,74175.99,99175.73,108765.04,81952.53,113708.04,92388.12,100579.09,47048.07,76536.02,79764.30,61761.44,9297.07,0.00,24227.89,35118.90,34942.13,41238.33,37826.63,36637.56,28978.45,25210.00,31416.99,26264.63,32226.04,5108.11,0.00,37204.35,63613.91,49712.61,27547.96,51354.02,40315.12,44560.57,17126.02,39047.64,42863.04,21375.85,38244.22,0.00,5197.13,7234.92,3617.32,6815.34,7460.24,8663.09,5197.07,8333.23,7894.41,3050.95,10878.66,9652.18,0.00,5008.86,2190.41,4913.80,4060.63,5643.48,2110.69,5167.59,5692.38,2460.04,7847.44,7961.48,3305.20,0.00,4946.27,2588.16,2418.82,3455.76,751.78,2627.38,3507.86,941.79,3188.86,3207.22,2281.65,2550.99,0.00,6001.2402,4102.6902,4628.4902,3604.7202,3009.9302,3788.6202,4092.2202,5408.3902,4775.6502,3997.2602,3254.9102,4592.8302,0.0000,35.410780,32.126251,50.319777,33.266452,39.656138,28.811605,53.583494,41.048633,32.927801,52.178252,54.943224,0.000000
12,100,2,2022,92386.05,74175.99,99175.73,108765.04,81952.53,113708.04,92388.12,100579.09,47048.07,76536.02,79764.30,61761.44,9297.07,39964.59,24227.89,35118.90,34942.13,41238.33,37826.63,36637.56,28978.45,25210.00,31416.99,26264.63,32226.04,5108.11,38172.38,37204.35,63613.91,49712.61,27547.96,51354.02,40315.12,44560.57,17126.02,39047.64,42863.04,21375.85,38244.22,8129.95,5197.13,7234.92,3617.32,6815.34,7460.24,8663.09,5197.07,8333.23,7894.41,3050.95,10878.66,9652.18,4367.00,5008.86,2190.41,4913.80,4060.63,5643.48,2110.69,5167.59,5692.38,2460.04,7847.44,7961.48,3305.20,1707.33,4946.27,2588.16,2418.82,3455.76,751.78,2627.38,3507.86,941.79,3188.86,3207.22,2281.65,2550.99,5884.0202,6001.2402,4102.6902,4628.4902,3604.7202,3009.9302,3788.6202,4092.2202,5408.3902,4775.6502,3997.2602,3254.9102,4592.8302,32.662712,35.410780,32.126251,50.319777,33.266452,39.656138,28.811605,53.583494,41.048633,32.927801,52.178252,54.943224
13,100,3,2022,122200.17,92386.05,74175.99,99175.73,108765.04,81952.53,113708.04,92388.12,100579.09,47048.07,76536.02,79764.30,61761.44,44443.41,39964.59,24227.89,35118.90,34942.13,41238.33,37826.63,36637.56,28978.45,25210.00,31416.99,

## Charges Prediction

In [195]:
# data.fillna(0)

In [196]:
# 'PRAC_CODE', 'MONTH', 'YEAR','PREV_MONTH_CHARGES','PREV_2ND_MONTH_CHARGES', 
#             'AGING_0_30','AGING_31_60','AGING_61_90','AGING_91_120',
#             'AGING_120_PLUS','CHARGES_SUM','next_month_charges'

In [197]:
# Predictors=['PRAC_CODE', 'MONTH', 'YEAR','PREV_MONTH_CHARGES','PREV_2ND_MONTH_CHARGES', 
#             'AGING_0_30','AGING_31_60','AGING_61_90','AGING_91_120',
#             'AGING_120_PLUS']
# TargetVariable=['CHARGES_SUM']
# X=data[Predictors].values
# y=data[TargetVariable].values

In [198]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [199]:
# xgb_regressor = xgb.XGBRegressor()

In [200]:
# xgb_regressor.fit(X_train,y_train)

In [201]:
# predictions = xgb_regressor.predict(X_test)

In [202]:
# predictions

In [203]:
# acc = np.mean(predictions/(predictions+y_test))*100

In [204]:
# acc

In [205]:
# pred = pd.DataFrame(predictions, columns = ['Pred_Charg'])
# pred['Pred_Charg'] = pred['Pred_Charg'].astype('int')

In [206]:
# orig = pd.DataFrame(y_test, columns = ['Test_Charg'])
# orig['Test_Charg'] = orig['Test_Charg'].astype('int')

In [207]:
# frames1 = [pred,orig]
# result = pd.concat(frames1, axis=1)

In [208]:
# result.sort_values(by = 'Test_Charg', ascending=False).reset_index(drop= True)

In [209]:
# data.loc[data['CHARGES_SUM'].between(41548664,44031225)]

In [210]:
# result['Difference'] = result['Pred_Charg']-result['Test_Charg']

In [211]:
# result.sort_values(by= 'Difference')

In [212]:
# predicted=result['Pred_Charg'].sum()
# original=result['Test_Charg'].sum()

# print(predicted,original)

# (predicted/original)*100
# # (original/predicted)*100

In [213]:
# predicted-original

In [214]:
# (predicted/(original+predicted))*100

## Payments Prediction 

In [215]:
data.head(1)

,PRAC_CODE,MONTH,YEAR,CHARG_SUM,PREV_M_CHARG,PREV_2ND_M_CHARG,PREV_3RD_M_CHARG,PREV_4TH_M_CHARG,PREV_5TH_M_CHARG,PREV_6TH_M_CHARG,PREV_7TH_M_CHARG,PREV_8TH_M_CHARG,PREV_9TH_M_CHARG,PREV_10TH_M_CHARG,PREV_11TH_M_CHARG,PREV_12TH_M_CHARG,PAY_SUM,PREV_M_PAY,PREV_2ND_M_PAY,PREV_3RD_M_PAY,PREV_4TH_M_PAY,PREV_5TH_M_PAY,PREV_6TH_M_PAY,PREV_7TH_M_PAY,PREV_8TH_M_PAY,PREV_9TH_M_PAY,PREV_10TH_M_PAY,PREV_11TH_M_PAY,PREV_12TH_M_PAY,AGING_0_30,PREV_M_AGING_0_30,PREV_2M_AGING_0_30,PREV_3M_AGING_0_30,PREV_4M_AGING_0_30,PREV_5M_AGING_0_30,PREV_6M_AGING_0_30,PREV_7M_AGING_0_30,PREV_8M_AGING_0_30,PREV_9M_AGING_0_30,PREV_10M_AGING_0_30,PREV_11M_AGING_0_30,PREV_12M_AGING_0_30,AGING_31_60,PREV_M_AGING_31_60,PREV_2M_AGING_31_60,PREV_3M_AGING_31_60,PREV_4M_AGING_31_60,PREV_5M_AGING_31_60,PREV_6M_AGING_31_60,PREV_7M_AGING_31_60,PREV_8M_AGING_31_60,PREV_9M_AGING_31_60,PREV_10M_AGING_31_60,PREV_11M_AGING_31_60,PREV_12M_AGING_31_60,AGING_61_90,PREV_M_AGING_61_90,PREV_2M_AGING_61_90,PREV_3M_AGING_61_90,PREV_4M_AGING_61_90,PREV_5M_AGING_61_90,PREV_6M_AGING_61_90,PREV_7M_AGING_61_90,PREV_8M_AGING_61_90,PREV_9M_AGING_61_90,PREV_10M_AGING_61_90,PREV_11M_AGING_61_90,PREV_12M_AGING_61_90,AGING_91_120,PREV_M_AGING_91_120,PREV_2M_AGING_91_120,PREV_3M_AGING_91_120,PREV_4M_AGING_91_120,PREV_5M_AGING_91_120,PREV_6M_AGING_91_120,PREV_7M_AGING_91_120,PREV_8M_AGING_91_120,PREV_9M_AGING_91_120,PREV_10M_AGING_91_120,PREV_11M_AGING_91_120,PREV_12M_AGING_91_120,AGING_120_PLUS,PREV_M_AGING_120_PLUS,PREV_2M_AGING_120_PLUS,PREV_3M_AGING_120_PLUS,PREV_4M_AGING_120_PLUS,PREV_5M_AGING_120_PLUS,PREV_6M_AGING_120_PLUS,PREV_7M_AGING_120_PLUS,PREV_8M_AGING_120_PLUS,PREV_9M_AGING_120_PLUS,PREV_10M_AGING_120_PLUS,PREV_11M_AGING_120_PLUS,PREV_12M_AGING_120_PLUS,PREV_M_PC_RATIO,PREV_2M_PC_RATIO,PREV_3M_PC_RATIO,PREV_4M_PC_RATIO,PREV_5M_PC_RATIO,PREV_6M_PC_RATIO,PREV_7M_PC_RATIO,PREV_8M_PC_RATIO,PREV_9M_PC_RATIO,PREV_10M_PC_RATIO,PREV_11M_PC_RATIO,PREV_12M_PC_RATIO
11,100,1,2022,74175.99,99175.73,108765.04,81952.53,113708.04,92388.12,100579.09,47048.07,76536.02,79764.3,61761.44,9297.07,0.0,24227.89,35118.9,34942.13,41238.33,37826.63,36637.56,28978.45,25210.0,31416.99,26264.63,32226.04,5108.11,0.0,37204.35,63613.91,49712.61,27547.96,51354.02,40315.12,44560.57,17126.02,39047.64,42863.04,21375.85,38244.22,0.0,5197.13,7234.92,3617.32,6815.34,7460.24,8663.09,5197.07,8333.23,7894.41,3050.95,10878.66,9652.18,0.0,5008.86,2190.41,4913.8,4060.63,5643.48,2110.69,5167.59,5692.38,2460.04,7847.44,7961.48,3305.2,0.0,4946.27,2588.16,2418.82,3455.76,751.78,2627.38,3507.86,941.79,3188.86,3207.22,2281.65,2550.99,0.0,6001.2402,4102.6902,4628.4902,3604.7202,3009.9302,3788.6202,4092.2202,5408.3902,4775.6502,3997.2602,3254.9102,4592.8302,0.0,35.41078,32.126251,50.319777,33.266452,39.656138,28.811605,53.583494,41.048633,32.927801,52.178252,54.943224,0.0


In [216]:
Predictors=['PRAC_CODE','MONTH','YEAR','CHARG_SUM','PREV_M_CHARG','PREV_2ND_M_CHARG','PREV_3RD_M_CHARG','PREV_4TH_M_CHARG',
'PREV_5TH_M_CHARG','PREV_6TH_M_CHARG','PREV_7TH_M_CHARG','PREV_8TH_M_CHARG','PREV_9TH_M_CHARG', 'PREV_10TH_M_CHARG',
'PREV_11TH_M_CHARG','PREV_12TH_M_CHARG','PREV_M_PAY','PREV_2ND_M_PAY','PREV_3RD_M_PAY','PREV_4TH_M_PAY','PREV_5TH_M_PAY',
'PREV_6TH_M_PAY','PREV_7TH_M_PAY', 'PREV_8TH_M_PAY','PREV_9TH_M_PAY','PREV_10TH_M_PAY','PREV_11TH_M_PAY',
'PREV_12TH_M_PAY','AGING_0_30','PREV_M_AGING_0_30','PREV_2M_AGING_0_30', 'PREV_3M_AGING_0_30',
'PREV_4M_AGING_0_30','PREV_5M_AGING_0_30','PREV_6M_AGING_0_30','PREV_7M_AGING_0_30', 'PREV_8M_AGING_0_30',
'PREV_9M_AGING_0_30','PREV_10M_AGING_0_30','PREV_11M_AGING_0_30','PREV_12M_AGING_0_30','AGING_31_60','PREV_M_AGING_31_60', 
'PREV_2M_AGING_31_60','PREV_3M_AGING_31_60','PREV_4M_AGING_31_60','PREV_5M_AGING_31_60', 'AGING_61_90',
'PREV_6M_AGING_31_60','PREV_7M_AGING_31_60','PREV_8M_AGING_31_60','PREV_9M_AGING_31_60','PREV_10M_AGING_31_60',
'PREV_11M_AGING_31_60','PREV_12M_AGING_31_60','AGING_61_90','PREV_M_AGING_61_90','PREV_2M_AGING_61_90','PREV_3M_AGING_61_90',
'PREV_4M_AGING_61_90','PREV_5M_AGING_61_90','PREV_6M_AGING_61_90','PREV_7M_AGING_61_90','PREV_8M_AGING_61_90', 
'PREV_9M_AGING_61_90','PREV_10M_AGING_61_90','PREV_11M_AGING_61_90','PREV_12M_AGING_61_90','AGING_91_120',
'PREV_M_AGING_91_120','PREV_2M_AGING_91_120','PREV_3M_AGING_91_120','PREV_4M_AGING_91_120','PREV_5M_AGING_91_120', 
'PREV_6M_AGING_91_120','PREV_7M_AGING_91_120','PREV_8M_AGING_91_120','PREV_9M_AGING_91_120','PREV_10M_AGING_91_120',
'PREV_11M_AGING_91_120','PREV_12M_AGING_91_120','AGING_120_PLUS','PREV_M_AGING_120_PLUS','PREV_2M_AGING_120_PLUS',
'PREV_3M_AGING_120_PLUS','PREV_4M_AGING_120_PLUS','PREV_5M_AGING_120_PLUS','PREV_6M_AGING_120_PLUS','PREV_7M_AGING_120_PLUS',
'PREV_8M_AGING_120_PLUS','PREV_9M_AGING_120_PLUS','PREV_10M_AGING_120_PLUS','PREV_11M_AGING_120_PLUS','PREV_12M_AGING_120_PLUS']
TargetVariable=['PAY_SUM']
X=data[Predictors].values
y=data[TargetVariable].values

In [217]:
# Predictors=['PRAC_CODE','MONTH','YEAR','CHARG_SUM','PREV_M_CHARG','PREV_2ND_M_CHARG','PREV_M_PAY','PREV_2ND_M_PAY',
# 'PREV_3RD_M_PAY','PREV_4TH_M_PAY','PREV_5TH_M_PAY','PREV_6TH_M_PAY','AGING_0_30','PREV_M_PC_RATIO',
# 'PREV_2M_PC_RATIO', 'PREV_3M_PC_RATIO', 'PREV_4M_PC_RATIO','PREV_5M_PC_RATIO','PREV_6M_PC_RATIO']
# TargetVariable=['PAY_SUM']
# X=data[Predictors].values
# y=data[TargetVariable].values

In [218]:
# Predictors=['PRACTICE_CODE', 'MONTH', 'YEAR','CHARGES_SUM','PREV_MONTH_CHARGES','PREV_2ND_MONTH_CHARGES',
#             'PREV_MONTH_PAYMENTS','PREV_2ND_MONTH_PAYMENTS','AGING_0_30','AGING_31_60','AGING_61_90','AGING_91_120',
#             'AGING_120_PLUS', 'PC_RATIO']
# TargetVariable=['PAYMNET_SUM']
# X=data[Predictors].values
# y=data[TargetVariable].values

In [219]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [220]:
X

array([[1.0000000e+02, 1.0000000e+00, 2.0220000e+03, ..., 3.2549102e+03,
        4.5928302e+03, 0.0000000e+00],
       [1.0000000e+02, 2.0000000e+00, 2.0220000e+03, ..., 3.9972602e+03,
        3.2549102e+03, 4.5928302e+03],
       [1.0000000e+02, 3.0000000e+00, 2.0220000e+03, ..., 4.7756502e+03,
        3.9972602e+03, 3.2549102e+03],
       ...,
       [9.0911050e+06, 1.1000000e+01, 2.0220000e+03, ..., 8.3881850e+04,
        7.0718980e+04, 7.1067980e+04],
       [9.0911050e+06, 1.2000000e+01, 2.0220000e+03, ..., 8.3983850e+04,
        8.3881850e+04, 7.0718980e+04],
       [9.0911050e+06, 1.0000000e+00, 2.0230000e+03, ..., 8.4015850e+04,
        8.3983850e+04, 8.3881850e+04]])

In [221]:
xgb_regressor = xgb.XGBRegressor(n_estimators=500)

In [222]:
xgb_regressor.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=500, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [223]:
predictions_pay = xgb_regressor.predict(X_test)

In [224]:
acc = np.mean(predictions_pay/(predictions_pay+y_test))*100
acc

50.77904317286301

In [225]:
pd.DataFrame(xgb_regressor.feature_importances_)

,0
0,1.533356e-05
1,8.804915e-03
2,2.522064e-03
3,9.678741e-01
4,1.711746e-05
5,1.103473e-05
6,2.351878e-05
7,1.427277e-05
8,6.744640e-05
9,6.311232e-06


In [226]:
pred_pay = pd.DataFrame(predictions_pay, columns = ['Pred_Pay'])
pred_pay['Pred_Pay'] = pred_pay['Pred_Pay'].astype('int')

In [227]:
orig_pay = pd.DataFrame(y_test, columns = ['Test_Pay'])
orig_pay['Test_Pay'] = orig_pay['Test_Pay'].astype('int')

In [228]:
frames = [pred_pay,orig_pay]
result_pay = pd.concat(frames, axis=1)

In [3]:
# result_pay.sort_values(by='Pred_Pay', ascending=False)

In [230]:
result_pay['Difference']= result_pay['Pred_Pay']-result_pay['Test_Pay']

In [2]:
# result_pay.head()

In [232]:
result_pay['VAR_PAY%']=(((result_pay['Test_Pay']-result_pay['Pred_Pay'])/result_pay['Pred_Pay'])*100).round(1)

In [233]:
result_pay['VAR_PAY%']=np.abs(result_pay['VAR_PAY%'])
result_pay['VAR_PAY%'].value_counts(bins=[0,5,10,15,20,1000],normalize=True)

(20.0, 1000.0]    0.488499
(-0.001, 5.0]     0.161622
(5.0, 10.0]       0.127724
(10.0, 15.0]      0.119855
(15.0, 20.0]      0.099274
Name: VAR_PAY%, dtype: float64

In [4]:
# result_pay

In [5]:
# result_pay.sort_values(by = 'VAR_PAY%')

In [236]:
predicted_pay=result_pay['Pred_Pay'].sum()
original_pay=result_pay['Test_Pay'].sum()

print(predicted_pay,original_pay)

# (predicted_pay/original_pay)*100
(original_pay/predicted_pay)*100

183671788 185092582


100.77355048125301

In [237]:
original_pay-predicted_pay

1420794

In [238]:
(predicted_pay/(original_pay+predicted_pay))*100

49.80735747328301

In [182]:
# # x = np.linspace(0, 2, 100)  # Sample data.
# # Note that even in the OO-style, we use `.pyplot.figure` to create the Figure.
# fig, ax = plt.subplots(figsize=(30, 6.7), layout='constrained')
# ax.plot(orig_pay, label='orig_pay')  # Plot some data on the axes.
# ax.plot(pred_pay, label='pred_pay')  # Plot more data on the axes...
# ax.set_xlabel('x label')  # Add an x-label to the axes.
# ax.set_ylabel('y label')  # Add a y-label to the axes.
# ax.set_title("Simple Plot")  # Add a title to the axes.
# ax.legend();  # Add a legend.

In [183]:
# plt.rcParams["figure.figsize"] = (30,10)
# #plt.plot(train.set_index('ds'),color='y', label='Train Data')
# #plt.plot(tc,color='b', label='In-Sample-Forecast')
# plt.plot(orig_pay,color='r', label='Forecast')
# plt.plot(orig_pay,color='g', label='Test Data')
# plt.xlabel("PRAC_CODE")
# plt.ylabel("PAYMNET_SUM")
# plt.title("RCM Claims Forecast")
# plt.legend(loc='upper right')
# plt.show()

In [ ]:
# data.loc[data['PAYMNET_SUM'].between(325,325)]